# Datashader: Census + Elevation

### Setup
The following imports will be needed to complete the exercises or provide for an improved notebook display:

In [1]:

from os import path

import numpy as np
import pandas as pd
import rasterio as rio
import datashader as ds
import datashader.transfer_functions as tf

from datashader.bokeh_ext import InteractiveImage

from bokeh.models import Range1d, VBox
from bokeh.plotting import Figure
from bokeh.io import output_notebook, show
from bokeh.tile_providers import STAMEN_TONER

import dask.dataframe as dd

output_notebook()

Loading BokehJS ...

### Load Elevation Data 

In [2]:
# load austin elevation data
path = './data/austin_dem.tif'
elevation_data = rio.open(path)

### Load Census Data

In [3]:
# first try loading from castra, but fallback to HDF5
try:
    census_df = dd.from_castra('./data/census.castra')
except:
    census_df = pd.read_hdf('./data/census.h5', 'census')
    census_df.race = census_df.race.astype('category')

In [4]:
def base_plot(tools='pan,wheel_zoom,reset',plot_width=900, plot_height=500, x_range=None, y_range=None, **plot_args):
    p = Figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        background_fill_color='black',
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)

    p.xgrid.grid_line_alpha = 0
    p.ygrid.grid_line_alpha = 0
    return p

In [5]:
w = np.ceil(elevation_data.width / 8.0).astype(np.int64)
h = np.ceil(elevation_data.height / 8.0).astype(np.int64)

In [6]:
xmin = elevation_data.bounds.left
ymin = elevation_data.bounds.bottom
xmax = elevation_data.bounds.right
ymax = elevation_data.bounds.top

In [7]:
cvs = ds.Canvas(plot_width=w,
                plot_height=h,
                x_range=(xmin, xmax),
                y_range=(ymin, ymax))
w,h,xmin,xmax, ymin,ymax

(871,
 506,
 -11020645.049574472,
 -10797986.371765953,
 3503546.483215993,
 3632767.180763634)

In [8]:
elevation_agg = cvs.raster(elevation_data)

ImportError: install rasterio and skimage to use this feature

In [ ]:
people = cvs.points(census_df, 'meterswest', 'metersnorth', ds.count_cat('race'))

In [ ]:
xs, ys = np.meshgrid(people.x_axis, people.y_axis)

# horizontal stack
summary_df = pd.DataFrame()
summary_df['x'] = xs.flatten()
summary_df['y'] = xs.flatten()
summary_df['elevation'] = elevation_agg.data.flatten()

cats = people[people.dims[2]].values.tolist()
for i, e in enumerate(cats):
    summary_df[str(e)] = people[:, :, i].data.flatten().astype(float)

In [ ]:
def update_image(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w,
                          plot_height=h,
                          x_range=x_range,
                          y_range=y_range)
    
    white_people = cvs.points(summary_df[(summary_df.w > summary_df.h) &
                                         (summary_df.w > summary_df.b)],
                              'x',
                              'elevation',
                              ds.sum('w'))
    
    hispanic_people = cvs.points(summary_df[(summary_df.h > summary_df.w) &
                                            (summary_df.h > summary_df.b)], 
                                 'x',
                                 'elevation',
                                 ds.sum('h'))
    
    black_people = cvs.points(summary_df[(summary_df.b > summary_df.w) & 
                                         (summary_df.b > summary_df.h)], 
                              'x',
                              'elevation',
                              ds.sum('b'))
    
    asian_people = cvs.points(summary_df[(summary_df.a > summary_df.w) & 
                                         (summary_df.a > summary_df.h)], 
                              'x',
                              'elevation',
                              ds.sum('a'))
    
    img = tf.stack(
            tf.shade(white_people, cmap=['black', 'cyan'], how='eq_hist', alpha=255),
            tf.shade(hispanic_people, cmap=['black', 'yellow'], how='eq_hist', alpha=255),
            tf.shade(black_people, cmap=['black', 'lime'], how='eq_hist', alpha=255),
            tf.shade(asian_people, cmap=['black', 'fuchsia'], how='eq_hist', alpha=255),
    )

    return tf.dynspread(img, .5, 10)

x_range = Range1d(start=-1.102e7, end=-1.0798e7)
elevation_plot = base_plot(x_range=x_range,
                           y_range=(100.0, 620.0),
                           title="Austin Area: Elevation from West to East symbolized by Race")
InteractiveImage(elevation_plot, update_image)